In [3]:
!pip install kafka-python

  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)

[notice] A new release of pip available: 22.2.2 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [8]:
from scipy.special import boxcox
import requests
from kafka import KafkaProducer
import pandas as pd
import json
import io

In [9]:
def zscore_normalization(df, name):
    mean = df[name].mean()
    sd = df[name].std()

    df[name] = (df[name] - mean) / sd

def preprocess(df):
    df = df.drop(columns=['Name', 'md5'])
    for i in df.columns:
        if i != 'legitimate':
            df[i] = boxcox(df[i], 0.5)
            zscore_normalization(df, i)
    correlation_matrix = df.corr()
    cols_to_drop = []
    for i in df.columns:
        for j in df.columns:
            if i != j and i != 'legitimate' and j != 'legitimate' and abs(correlation_matrix[i][j]) > 0.6 and i not in cols_to_drop and j not in cols_to_drop:
                cols_to_drop.append(i)
    cols_to_drop = set(cols_to_drop)
    df.drop(columns=cols_to_drop, inplace=True)
    return df

In [14]:
import logging
import requests
import pandas as pd
from kafka import KafkaProducer
import json

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Broker address
brokers = ["ec2-107-21-74-54.compute-1.amazonaws.com:9092"]

# GitHub raw URL for the CSV file
github_url = "https://raw.githubusercontent.com/tsimhadri-ews/internproject/malware-detection-0/src/MalwareData.csv"

# Download the CSV file from GitHub
logger.info("Downloading CSV file from GitHub...")
response = requests.get(github_url)

# Check if the request was successful
if response.status_code == 200:
    logger.info("CSV file downloaded successfully.")
    # Read the data into a pandas DataFrame
    df = pd.read_csv(github_url, sep='|')
    df = preprocess(df)
    df = df.drop(columns=['legitimate'])
    logger.info("CSV file processed successfully.")
else:
    logger.error(f"Failed to download file: {response.status_code}")
    exit()


topic = "three"

# Create a Kafka producer
logger.info("Creating Kafka producer...")
producer = KafkaProducer(
    bootstrap_servers=brokers,
    value_serializer=lambda message: json.dumps(message).encode('utf-8'),
)
# Get the first 5 rows in the DataFrame
first_5_rows = df.head(5)

# Convert each row to a dictionary and send to Kafka
for index, row in first_5_rows.iterrows():
    data = row.to_dict()
    producer.send(topic, data)
    logger.info(f"Sent row {index + 1} to Kafka")

# Flush data and close the producer
producer.flush()
producer.close()

logger.info(f"First 5 rows sent to Kafka topic: {topic}")


INFO:__main__:Downloading CSV file from GitHub...
INFO:__main__:CSV file downloaded successfully.
INFO:__main__:CSV file processed successfully.
INFO:__main__:Creating Kafka producer...
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=ec2-107-21-74-54.compute-1.amazonaws.com:9092 <connecting> [IPv4 ('107.21.74.54', 9092)]>: connecting to ec2-107-21-74-54.compute-1.amazonaws.com:9092 [('107.21.74.54', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=ec2-107-21-74-54.compute-1.amazonaws.com:9092 <connecting> [IPv4 ('107.21.74.54', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:__main__:Sent row 1 to Kafka
INFO:__main__:Sent row 2 to Kafka
INFO:__main__:Sent row 3 to Kafka
INFO:__main__:Sent row 4 to Kafka
INFO:__main__:Sent row 5 to Kafka
INFO:kafka.conn:<Broker

In [14]:
github_url = "https://raw.githubusercontent.com/tsimhadri-ews/internproject/malware-detection-0/src/MalwareData.csv"

response = requests.get(github_url)

# Check if the request was successful
if response.status_code == 200:
    #logger.info("CSV file downloaded successfully.")
    # Read the data into a pandas DataFrame
    df = pd.read_csv(github_url, sep='|')
    df = preprocess(df)

In [15]:
df.dtypes

SizeOfOptionalHeader           float64
MinorLinkerVersion             float64
SizeOfCode                     float64
SizeOfInitializedData          float64
SizeOfUninitializedData        float64
BaseOfCode                     float64
BaseOfData                     float64
ImageBase                      float64
SectionAlignment               float64
MajorOperatingSystemVersion    float64
MinorImageVersion              float64
MajorSubsystemVersion          float64
SizeOfHeaders                  float64
CheckSum                       float64
Subsystem                      float64
DllCharacteristics             float64
SizeOfStackReserve             float64
SizeOfStackCommit              float64
SizeOfHeapReserve              float64
NumberOfRvaAndSizes            float64
SectionsNb                     float64
SectionsMinEntropy             float64
SectionsMaxEntropy             float64
SectionMaxRawsize              float64
SectionsMinVirtualsize         float64
SectionMaxVirtualsize    